In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
import time
import numpy as np

warnings.simplefilter('ignore')

def get_shareholding(x):
    url = "https://www.screener.in/company/" + x +"/"
    response = requests.get(url)

    soup = bs(response.content, "html.parser")

    quarterly_shp_section = soup.find('div', {'id': 'quarterly-shp'})

    shareholding_table = quarterly_shp_section.find('table', {'class': 'data-table'})
    shareholding_data = []
    for row in shareholding_table.find_all('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        shareholding_data.append(cols)

    df = pd.DataFrame(shareholding_data)
    time.sleep(4);
    return df


In [2]:

def HoldingScore(ticker):
    try:
        df = get_shareholding(ticker)
        if len(df[0].to_numpy()) == 6:
            df[0] = ["x","Promoter", "FII", "DII", "Retail","Share Holders"]
        if len(df[0].to_numpy()) == 7:
            df[0] = ["x","Promoter", "FII", "DII","Gov", "Retail","Share Holders"]
        if len(df[0].to_numpy()) == 8:
            df[0] = ["x","Promoter", "FII", "DII","Gov", "Retail","other","Share Holders"]
        if len(df[0].to_numpy()) > 8:
            df[0] = ["x","Promoter", "FII", "DII","Gov", "Retail","other","Share Holders",'xx']
        
        df = df.transpose()

        df = df.drop(columns=[0])
        df.columns = df.iloc[0]
        df = df.iloc[1:]

        df.drop([1, 2, 3, 4, 5, 6, 7, 8], inplace=True)
        # Convert percentage strings to float
        df['Promoter'] = df['Promoter'].str.rstrip('%').astype(float)
        df['FII'] = df['FII'].str.rstrip('%').astype(float)
        df['DII'] = df['DII'].str.rstrip('%').astype(float)
        df['Retail'] = df['Retail'].str.rstrip('%').astype(float)
        try:
            df['Gov'] = df['Gov'].str.rstrip('%').astype(float)
        except:
            pass
        # Remove commas and convert 'share holders' to int
        df['Share Holders'] = df['Share Holders'].str.replace(',', '').astype(int)
        score = round((np.diff(df['FII'].to_numpy()) / df['FII'].to_numpy()[:-1]).sum() + (np.diff(df['DII'].to_numpy()) / df['DII'].to_numpy()[:-1]).sum() + (np.diff(df['Promoter'].to_numpy()) / df['Promoter'].to_numpy()[:-1]).sum() - (np.diff(df['Retail'].to_numpy()) / df['Retail'].to_numpy()[:-1]).sum(),5)
        holding = round(-(np.diff(df['Share Holders'].to_numpy()) / df['Share Holders'].to_numpy()[:-1]).sum(),3)
        return score, holding
    except Exception as e:
        print('error '+ticker)
        print(e)
        return 0, 0
        
def csv_caller(x):
    of = pd.read_csv(x)
    of = of.drop(columns=['Unnamed: 0'])
    of["Holding score"] = 0
    of['Holding no'] = 0
    k = 0
    for i in of['nsecode']:
        Score, holding = HoldingScore(i)
        of['Holding score'][k] = Score
        of['Holding no'][k] = holding
        k+=1
    of.fillna(0.0, inplace=True)
    return of

mid_ga = csv_caller("long_term.csv")
short_ga = csv_caller("short_term.csv")
long_ga = csv_caller("mid_term.csv")


error KITEX
'Promoter'
error CONSOFINVT
'NoneType' object has no attribute 'find_all'
error KITEX
'Promoter'
error CONSOFINVT
'NoneType' object has no attribute 'find_all'
error GATEWAY
'[7, 8] not found in axis'


In [3]:
mid_ga['cat'] = "M"
short_ga['cat'] = "S"
long_ga['cat'] = "L"

In [4]:
mid_ga

,nsecode,Holding score,Holding no,cat
0,BAJFINANCE,0.06494,-0.036,M
1,BAJAJ-AUTO,0.01031,0.102,M
2,ROLEXRINGS,0.00000,-0.000,M
3,ECLERX,0.16263,-0.398,M
4,M&M,0.00255,0.019,M
5,INDUSINDBK,0.02296,-0.022,M
6,SHAREINDIA,0.45801,0.049,M
7,BHARTIARTL,0.08108,0.091,M
8,ICICIPRULI,0.18346,0.007,M
9,BALRAMCHIN,0.15363,0.146,M


In [5]:
short_ga

,nsecode,Holding score,Holding no,cat
0,BAJFINANCE,0.06494,-0.036,S
1,BAJAJ-AUTO,0.01031,0.102,S
2,ROLEXRINGS,0.00000,-0.000,S
3,SHRIRAMFIN,-1.32044,-0.006,S
4,PHOENIXLTD,0.12642,-0.441,S
5,ECLERX,0.16263,-0.398,S
6,M&M,0.00255,0.019,S
7,ACCELYA,-1.53982,-0.075,S
8,INDUSINDBK,0.02296,-0.022,S
9,SHAREINDIA,0.45801,0.049,S


In [6]:
merged_df = pd.concat([mid_ga, short_ga, long_ga], axis=0)

In [7]:
merged_df

,nsecode,Holding score,Holding no,cat
0,BAJFINANCE,0.06494,-0.036,M
1,BAJAJ-AUTO,0.01031,0.102,M
2,ROLEXRINGS,0.00000,-0.000,M
3,ECLERX,0.16263,-0.398,M
4,M&M,0.00255,0.019,M
...,...,...,...,...
1,BAJAJFINSV,0.43342,-0.226,L
2,SHAREINDIA,0.45801,0.049,L
3,AMBUJACEM,-0.09656,-0.322,L
4,ZUARI,-0.32538,0.015,L


In [8]:
merged_df = merged_df.sort_values(by='Holding score', ascending=False)

In [12]:
import streamlit as st
import base64

In [14]:
from datetime import datetime
today_date = datetime.today().strftime('%d-%m-%Y')

In [16]:
merged_df.to_csv(today_date+'.csv')

In [ ]:
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # Base64 encoding
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}.csv">{text}</a>'
    return href

In [ ]:
# Streamlit app
st.title('Display DataFrame in Sidebar')

# Display DataFrame in the sidebar
st.sidebar.write('### DataFrame in Sidebar')
st.sidebar.write(merged_df)